<a href="https://colab.research.google.com/github/eduardo-eal/TCC/blob/main/TCC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FAKE NEWS

Instalar bibliotecas

In [ ]:
!pip install texthero
!pip install -U spacy

     |████████████████████████████████| 1.5 MB 19.7 MB/s 
     |████████████████████████████████| 235 kB 61.7 MB/s 
     |████████████████████████████████| 748 kB 43.4 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 6.0 MB 14.6 MB/s 
     |████████████████████████████████| 10.1 MB 50.1 MB/s 
     |████████████████████████████████| 181 kB 52.3 MB/s 
     |████████████████████████████████| 451 kB 49.8 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 628 kB 51.0 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attemp

Importar bibliotecas necessárias

In [ ]:
import pandas as pd
import texthero as hero

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/spacy/util.py:841: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.2.1,<3.3.0
  warnings.warn(warn_msg)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Importar Dataset de notícias verdadeiras e falsas

In [ ]:
#arquivos podem ser obtidos em https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset
noticias_verdadeiras = pd.read_csv('/content/drive/MyDrive/PUC/16 - Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina (2021)/True.csv');
noticias_falsas = pd.read_csv('/content/drive/MyDrive/PUC/16 - Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina (2021)/Fake.csv');

Primeiras notícias verdadeiras

In [ ]:
noticias_verdadeiras.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [ ]:
noticias_verdadeiras.loc[7,'text']

'The following statements\xa0were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own.\xa0Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - While the Fake News loves to talk about my so-called low approval rating, @foxandfriends just showed that my rating on Dec. 28, 2017, was approximately the same as President Obama on Dec. 28, 2009, which was 47%...and this despite massive negative Trump coverage & Russia hoax! [0746 EST] - Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE! [0804 EST] -- Source link: (bit.ly/2jBh4LU) (bit.ly/2jpEXYR) '

Primeiras notícias falsas

In [ ]:
noticias_falsas.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


Em uma primeira análise já é possível identificar um padrão no início dos textos das notícias verdadeiras, com o nome da cidade seguido de "(Reuters)" que não é observado nas falsas, provavelmente devido à diferença de fontes das quais foram extraídas. Teremos que tratar/remover isso para não gerar um viés para as notícias verdadeiras.

Vamos criar uma nova coluna com a classe e juntar as bases para fazer um pré-processamento para todo o Dataset

In [ ]:
noticias_verdadeiras['fake']=0
noticias_falsas['fake']=1
noticias = pd.concat([noticias_verdadeiras,noticias_falsas])

In [ ]:
noticias.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


Embaralhar as notícias verdadeiras e falsas

In [ ]:
noticias = noticias.sample(frac=1.0).reset_index(drop=True)

noticias.head()

,title,text,subject,date,fake
0,Trump Treasury candidate's bank accused of dis...,WASHINGTON (Reuters) - Two California housing ...,politicsNews,"November 17, 2016",0
1,Ukraine drops tax probe of finance minister: f...,KIEV (Reuters) - Ukrainian prosecutors have en...,worldnews,"September 5, 2017",0
2,Use it or lose it: Occasional Ohio voters may ...,CINCINNATI (Reuters) - When Larry Harmon tried...,politicsNews,"June 2, 2016",0
3,WOW! HILLARY SPILLS THE BEANS On Funding Terro...,Hillary Clinton tells about funding the terror...,politics,"Aug 13, 2016",1
4,YOUR BLOOD WILL BOIL When You See Why College ...,This is how coddled kids who have never had to...,left-news,"Sep 12, 2016",1


In [ ]:
noticias.loc[2,'text']

'CINCINNATI (Reuters) - When Larry Harmon tried to vote on a marijuana initiative in November in his hometown of Kent, Ohio, the 59-year-old software engineer found his name had been struck from the voter rolls. Two hours south in Zanesville, restaurant worker Chris Conrad, 37, was also told he was no longer registered. Both men later found out why: they had not voted often enough. As the Nov. 8 elections loom, officials in Ohio have removed tens of thousands of voters from registration lists because they have not cast a ballot since 2008.  All U.S. states periodically cleanse their voter rolls, but only a handful remove voters simply because they don’t vote on a regular basis. And nowhere could the practice have a greater potential impact in the state-by-state battle for the White House than Ohio, a swing state that has backed the winner in every presidential election since 1960. Voters of all stripes in Ohio are affected, but the policy appears to be helping Republicans in the state’

Verificar campos nulos - Nenhum campo nulo no dataset

In [ ]:
noticias.isnull().sum()

title      0
text       0
subject    0
date       0
fake       0
dtype: int64

#Pré-processamento do título e texto

Vamos usar a biblioteca texthero para fazer uma limpeza no texto, retirando caracteres sem importância para a identificação de padrões e assim deixar o texto mais significativo para os algoritmos de IA.

In [ ]:
#Alterar todas as palavras para minúsculas
noticias['text'] = hero.lowercase(noticias['text'])

noticias['title'] = hero.lowercase(noticias['title'])

#Remover pontuação
noticias['text'] = hero.remove_punctuation(noticias['text'])

noticias['title'] = hero.remove_punctuation(noticias['title'])

#Remover acentos
noticias['text'] = hero.remove_diacritics(noticias['text'])

noticias['title'] = hero.remove_diacritics(noticias['title'])

#Remover dígitos
noticias['text'] = hero.remove_digits(noticias['text'])

noticias['title'] = hero.remove_digits(noticias['title'])


#Remover chaves, parênteses, colchetes...
noticias['text'] = hero.remove_brackets(noticias['text'])
noticias['text'] = hero.remove_angle_brackets(noticias['text'])
noticias['text'] = hero.remove_curly_brackets(noticias['text'])
noticias['text'] = hero.remove_round_brackets(noticias['text'])
noticias['text'] = hero.remove_square_brackets(noticias['text'])

noticias['title'] = hero.remove_brackets(noticias['title'])
noticias['title'] = hero.remove_angle_brackets(noticias['title'])
noticias['title'] = hero.remove_curly_brackets(noticias['title'])
noticias['title'] = hero.remove_round_brackets(noticias['title'])
noticias['title'] = hero.remove_square_brackets(noticias['title'])

#remover stopwords
noticias['text'] = hero.remove_stopwords(noticias['text'])

noticias['title'] = hero.remove_stopwords(noticias['title'])

#remover URL
noticias['text'] = hero.remove_urls(noticias['text'])

noticias['title'] = hero.remove_urls(noticias['title'])

#remover espaços em branco
noticias['text'] = hero.remove_whitespace(noticias['text'])

noticias['title'] = hero.remove_whitespace(noticias['title'])




Remover do título e texto palavas com menos de 3 caracteres e termo reuters (presente na maioria das notícias verdadeiras)

In [ ]:
noticias['text'] = noticias['text'].apply(lambda x: ' '.join([termo for termo in x.split() if len(termo) > 2 and termo != 'reuters']))
noticias['title'] = noticias['title'].apply(lambda x: ' '.join([termo for termo in x.split() if len(termo) > 2 and termo != 'reuters']))

In [ ]:
hero.top_words(noticias['text'])[:20]

trump         134052
said          132825
president      55893
would          55167
people         41857
one            37927
state          34488
also           31357
new            31311
clinton        28695
obama          28203
donald         28137
government     28048
house          27752
states         26845
republican     25568
year           24998
could          23957
united         23601
told           23367
Name: text, dtype: int64

Após limpeza do texto, ainda não existem campos nulos, mas o texto de 632 registros ficaram vazios

In [ ]:
noticias.isnull().sum()

title      0
text       0
subject    0
date       0
fake       0
dtype: int64

In [ ]:
#Textos vazios
(noticias['text'] == '').sum()

632

In [ ]:
noticias_sem_texto = noticias[noticias['text'] == '']

In [ ]:
noticias_sem_texto.groupby('fake').count()

,title,text,subject,date
fake,,,,
0,1,1,1,1
1,631,631,631,631


In [ ]:
#Títulos vazios - nenhum registro
(noticias['title'] == '').sum()

0

In [ ]:
#Título das notícias verdadeiras sem texto
noticias_sem_texto[noticias_sem_texto['fake']==0]['title']

31611    graphic supreme court roundup
Name: title, dtype: object

In [ ]:
#Título das notícias falsas sem texto
noticias_sem_texto[noticias_sem_texto['fake']==1]['title']

55       god made patriot need man knows fragility libe...
187      democrat operatives caught planning bully wome...
274      unreal cnn anchor claims trump committed "trea...
309      breaking donald trump video statement leaked p...
320      raw video shocking tour detroit ghetto...ruled...
                               ...                        
44444    two top media leftists destroy clintons spent ...
44594    nancy pelosi tries lead democrats chants obama...
44715    breaking ford announces million investment...j...
44869    florida sheriff hysterically shames woman bust...
44892    video trey gowdy gets real community policing ...
Name: title, Length: 631, dtype: object

Como existem 632 registros com texto em branco mas com o título contendo informação significativa, vamos concatenar título e texto para tratar esses dois campos como um único.

In [ ]:
#campo noticia_tratada vai conter o título seguido do texto
noticias['noticia_tratada'] = noticias['title']+' '+noticias['text']

In [ ]:
noticias.head()

,title,text,subject,date,fake,noticia_tratada
0,trump treasury candidate bank accused discrimi...,washington two california housing advocacy gro...,politicsNews,"November 17, 2016",0,trump treasury candidate bank accused discrimi...
1,ukraine drops tax probe finance minister finan...,kiev ukrainian prosecutors ended inquiry finan...,worldnews,"September 5, 2017",0,ukraine drops tax probe finance minister finan...
2,use lose occasional ohio voters may shut november,cincinnati larry harmon tried vote marijuana i...,politicsNews,"June 2, 2016",0,use lose occasional ohio voters may shut novem...
3,wow hillary spills beans funding terrorists......,hillary clinton tells funding terrorists would...,politics,"Aug 13, 2016",1,wow hillary spills beans funding terrorists......
4,blood boil see college students forced stand g...,coddled kids never deal losing innocent family...,left-news,"Sep 12, 2016",1,blood boil see college students forced stand g...


##Tokenizar e fazer lemmatization
Para preparar o texto para uso nos algoritmos de IA temos que tokenizar (dividir o texto em palavras) e em seguida vamos aplicar o lemmatization. Optamos por usar Lemmatization e não stemming pois lemmatization reduz as palavras a uma ainda existente na linguagem. O Stemming reduz a palavra a uma raiz não necessariamente uma palavra válida na linguagem, o que poderia dificultar nossa análise nos próximos passos.

In [ ]:
#tokenização e lemmatização com wordnet está rápida (pouco mais de 1min) mas deixando muitas palavras como verbos e conjugações diferentes
#import nltk
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('omw-1.4')

#lemmatizer = nltk.stem.WordNetLemmatizer()

#def f_lemmatize(texto):
#    return [lemmatizer.lemmatize(termo) for termo in nltk.word_tokenize(texto)]

#noticias['noticia_tratada_lemma'] = noticias['noticia_tratada'].apply(f_lemmatize)

In [ ]:
#tokenização e lematização com spacy está levando 30 min mas o resultado está melhor processado que com Wordnet
import spacy
nlp = spacy.load('en_core_web_sm')

def f_lemmatize(texto):
    return [termo.lemma_ for termo in nlp(texto)]

noticias['noticia_tratada_lemma'] = noticias['noticia_tratada'].apply(f_lemmatize)

In [ ]:
noticias.head()

,title,text,subject,date,fake,noticia_tratada,noticia_tratada_lemma
0,trump treasury candidate bank accused discrimi...,washington two california housing advocacy gro...,politicsNews,"November 17, 2016",0,trump treasury candidate bank accused discrimi...,"[trump, treasury, candidate, bank, accuse, dis..."
1,ukraine drops tax probe finance minister finan...,kiev ukrainian prosecutors ended inquiry finan...,worldnews,"September 5, 2017",0,ukraine drops tax probe finance minister finan...,"[ukraine, drop, tax, probe, finance, minister,..."
2,use lose occasional ohio voters may shut november,cincinnati larry harmon tried vote marijuana i...,politicsNews,"June 2, 2016",0,use lose occasional ohio voters may shut novem...,"[use, lose, occasional, ohio, voter, may, shut..."
3,wow hillary spills beans funding terrorists......,hillary clinton tells funding terrorists would...,politics,"Aug 13, 2016",1,wow hillary spills beans funding terrorists......,"[wow, hillary, spill, bean, fund, terrorist, ...."
4,blood boil see college students forced stand g...,coddled kids never deal losing innocent family...,left-news,"Sep 12, 2016",1,blood boil see college students forced stand g...,"[blood, boil, see, college, student, force, st..."


In [ ]:
noticias.drop(columns=['noticia_tratada'], inplace=True)


In [ ]:
#Gravar o dataset processado até o momento
noticias.to_csv('/content/drive/MyDrive/PUC/16 - Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina (2021)/noticias_preprocessadas.csv', index=False)
#Arquivo disponível em https://drive.google.com/file/d/1GDt17BwSrxJF5WJ0F8c0il6nO8xD7wCh/view?usp=sharing